In [1]:
import pandas as pd
import numpy as np
import json
import xgboost as xgb
import sklearn
import pickle
import time

In [2]:
df_test = pd.read_csv('test.csv')

In [3]:
df_test.columns.to_series().groupby(df_test.dtypes).groups

{int64: ['Id', 'MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold'], float64: ['LotFrontage', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt', 'GarageCars', 'GarageArea'], object: ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'Garag

In [4]:
df_test.drop('Id', axis=1,inplace=True) 

In [5]:
df_test['HasCentralAir'] = np.where(df_test['CentralAir']=='Y',1,0)
df_test.drop('CentralAir',axis=1,inplace=True)

In [6]:
# # Importo el diccionario de encodeo

In [7]:
with open('encoding_dict.txt', 'r') as file:
    encoding_dict = json.load(file)

In [8]:
encoding_dict

{'MSZoning': {'C (all)': 74528.0,
  'FV': 214014.06153846154,
  'RH': 131558.375,
  'RL': 191004.99478714162,
  'RM': 126316.83027522935},
 'Street': {'Grvl': 130190.5, 'Pave': 181130.53851444292},
 'Alley': {'Grvl': 122219.08, 'Pave': 168000.58536585365},
 'LotShape': {'IR1': 206101.6652892562,
  'IR2': 239833.36585365853,
  'IR3': 216036.5,
  'Reg': 164754.81837837838},
 'LandContour': {'Bnk': 143104.07936507938,
  'HLS': 231533.94,
  'Low': 203661.11111111112,
  'Lvl': 180183.74675819985},
 'LotConfig': {'Corner': 181623.4258555133,
  'CulDSac': 223854.6170212766,
  'FR2': 177934.5744680851,
  'FR3': 208475.0,
  'Inside': 176938.0475285171},
 'LandSlope': {'Gtl': 179956.7995658466,
  'Mod': 196734.13846153847,
  'Sev': 204379.23076923078},
 'Neighborhood': {'Blmngtn': 194870.88235294117,
  'Blueste': 137500.0,
  'BrDale': 104493.75,
  'BrkSide': 124834.05172413793,
  'ClearCr': 212565.42857142858,
  'CollgCr': 197965.77333333335,
  'Crawfor': 210624.72549019608,
  'Edwards': 128219.

In [9]:
# Encodeo variables del DataFrame a  scorear

In [10]:
for var_name in df_test.columns:
    var_dict_val = encoding_dict.get(var_name)
    df_test[var_name].replace(var_dict_val, inplace=True)    

In [11]:
df_test.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,HasCentralAir
0,20,131558.375000,80.0,11622,181130.538514,NaN,164754.818378,180183.746758,180950.95682,176938.047529,...,0,NaN,148751.089172,NaN,0,6,2010,173401.836622,175202.219533,1
1,20,191004.994787,81.0,14267,181130.538514,NaN,206101.665289,180183.746758,180950.95682,181623.425856,...,0,NaN,NaN,170750.0,12500,6,2010,173401.836622,175202.219533,1
2,60,191004.994787,74.0,13830,181130.538514,NaN,206101.665289,180183.746758,180950.95682,176938.047529,...,0,NaN,148751.089172,NaN,0,3,2010,173401.836622,175202.219533,1
3,60,191004.994787,78.0,9978,181130.538514,NaN,206101.665289,180183.746758,180950.95682,176938.047529,...,0,NaN,NaN,NaN,0,6,2010,173401.836622,175202.219533,1
4,120,191004.994787,43.0,5005,181130.538514,NaN,206101.665289,231533.940000,180950.95682,176938.047529,...,0,NaN,NaN,NaN,0,1,2010,173401.836622,175202.219533,1


In [12]:
filename = 'model_pickle.dat'
loaded_model = pickle.load(open(filename, 'rb'))
print(type(loaded_model))

<class 'xgboost.core.Booster'>


In [13]:
loaded_model

cols_when_model_builds = loaded_model.get_booster().feature_names

In [14]:
df_test.describe()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,HasCentralAir
count,1459.000000,1455.000000,1459.000000,1459.000000,1459.000000,107.000000,1459.000000,1459.000000,1.457000e+03,1459.000000,...,1459.000000,3.000000,290.000000,51.000000,1459.000000,1459.000000,1459.000000,1458.000000,1459.000000,1459.000000
mean,57.378341,179806.722628,68.706648,9819.161069,180921.052407,138050.067837,180482.928835,181661.243833,1.809510e+05,180613.421121,...,1.744345,393996.666667,153166.186071,150095.689476,58.167923,6.104181,2007.769705,180526.668551,180528.088466,0.930775
std,42.746880,27607.779632,22.309312,4955.517327,3261.078802,21877.370884,21589.391576,13579.697780,2.911382e-11,10993.656740,...,30.491646,166282.651029,13458.566125,12351.599063,630.806978,2.722432,1.301740,29252.071128,28940.570153,0.253924
min,20.000000,74528.000000,21.000000,1470.000000,130190.500000,122219.080000,164754.818378,143104.079365,1.809510e+05,176938.047529,...,0.000000,201990.000000,134286.363636,94000.000000,0.000000,1.000000,2006.000000,119850.000000,104125.000000,0.000000
25%,20.000000,191004.994787,59.000000,7391.000000,181130.538514,122219.080000,164754.818378,180183.746758,1.809510e+05,176938.047529,...,0.000000,345995.000000,148751.089172,151187.612245,0.000000,4.000000,2007.000000,173401.836622,175202.219533,1.000000
50%,50.000000,191004.994787,68.000000,9399.000000,181130.538514,122219.080000,164754.818378,180183.746758,1.809510e+05,176938.047529,...,0.000000,490000.000000,148751.089172,151187.612245,0.000000,6.000000,2008.000000,173401.836622,175202.219533,1.000000
75%,70.000000,191004.994787,80.000000,11517.500000,181130.538514,168000.585366,206101.665289,180183.746758,1.809510e+05,177934.574468,...,0.000000,490000.000000,148751.089172,151187.612245,0.000000,8.000000,2009.000000,173401.836622,175202.219533,1.000000
max,190.000000,214014.061538,200.000000,56600.000000,181130.538514,168000.585366,239833.365854,231533.940000,1.809510e+05,223854.617021,...,800.000000,490000.000000,178927.457627,170750.000000,17000.000000,12.000000,2010.000000,274945.418033,272291.752000,1.000000


In [15]:
df_test = df_test[['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'HasCentralAir', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition', 'Utilities', 'PavedDrive']]

In [16]:
dscore = xgb.DMatrix(df_test)

In [17]:
test_preds = loaded_model.predict(dscore)

In [19]:
df_test_preds = pd.DataFrame(data=test_preds)

In [22]:
df_test_preds.index = [1461 + x for x in range(0, len(df_test_preds))]

In [ ]:
timestr = time.strftime("%Y_%m_%d-%H_%M_%S")

In [24]:
df_test_preds.to_csv(f'xgboost_preds_tuned_{timestr}.csv')